# Kandidaten

In [1]:
import codecs
from collections import Counter
import nltk
import numpy as np
import os
import pandas as pd
import re
import scipy
import spacy

In [2]:
df = pd.read_csv('Korpus/corpus_lemmatized.csv', sep=',', encoding='utf-8')

In [3]:
df.head()

,doc,source,author,title,year,period,text_type,text,words,lemmatized_text
0,Espositivi.IV.4.Testo.txt,MIDIA,Ludovico Antonio Muratori,Antichità italiane,1700.0,1700-1750,espositivo,"﻿IV. 4. Ludovico Antonio Muratori, Antichità i...",8990.0,"﻿iv . 4 . Ludovico Antonio muratori , antichit..."
1,Poesia.IV.11.Testo.txt,MIDIA,Giambattista Felice Zappi,Poesie,1700.0,1700-1750,poesia,IV. 11. Rime degli arcadi: Tirsi Leucasio (Gio...,6113.0,iv . 11 . rima del arcade : tirsi leucasio ( g...
2,Personali.IV.15.Testo.txt,MIDIA,Pietro Giannone,Vita scritta da lui medesimo,1700.0,1700-1750,personale,[Proemio]\nPrendo a scrivere la mia vita e qua...,10118.0,[ proemio ] \n prendere a scrivere il mio vita...
3,Personali.IV.4.Testo.txt,MIDIA,Vincenzo da Filicaia,Lettere inedite a Lorenzo Magalotti,1700.0,1700-1750,personale,"IV. 4. Vincenzo da Filicaia, Lettere inedite a...",10073.0,"iv . 4 . Vincenzo da filicaia , lettere inedit..."
4,Poesia.IV.4.Testo.txt,MIDIA,Faustina Maratti Zappi,Poesie,1700.0,1700-1750,poesia,IV. 4. Rime degli Arcadi: Aglauro Cidonia (Fau...,3184.0,iv . 4 . rima del arcadi : aglauro cidonia ( f...


In [4]:
df.shape

(239270, 10)

In [5]:
df.text = df.text.fillna('')
df.lemmatized_text = df.lemmatized_text.fillna('')

In [6]:
# Teilframes pro Zeitraum erstellen

period_frames = []
periods = list(df['period'].unique())
df_periods = dict(tuple(df.groupby(by='period')))

for df, period in zip(df_periods, periods):
    df_period = df_periods[period]
    period_frames.append(df_period)

In [7]:
# Funktion, die bestimmte Wörter zählt (im lemmatisierten Text)

def count_keywords(df, keywords):
    keyword_dict = {w:w.replace(' ', '_') for w in keywords}

    corpus = ' '.join(df.lemmatized_text).lower()
    for w,w2 in keyword_dict.items():
        corpus = corpus.replace(w,w2)

    all_counts = Counter(corpus.split())
    final_counts = {w:all_counts[w2] for w,w2 in keyword_dict.items()}
    return final_counts

In [8]:
# mögliche Kandidaten

keywords = ['atrocemente', 'formidabilmente', 'mostruosamente', 'orrendamente', 
            'orribilmente', 'spaventosamente', 'terribilmente', 'tremendamente', 
            'molto', 'angoscia', 'angosciare', 'ansia', 'atterrire', 'panico', 
            'paura', 'raccapricciare', 'raccapriccio', 'schifo', 'sgomentare', 
            'sgomento', 'spaventare', 'spavento', 'terrore', 'domani', 'finora', 
            'ieri', 'oggi', 'presto', 'raramente', 'sempre', 'spesso']

In [9]:
candidates = {}

for df, period in zip(period_frames, periods):
    res = count_keywords(df, keywords)
    candidates[period] = res

In [10]:
candidates

{'1700-1750': {'atrocemente': 2,
  'formidabilmente': 0,
  'mostruosamente': 2,
  'orrendamente': 1,
  'orribilmente': 28,
  'spaventosamente': 13,
  'terribilmente': 17,
  'tremendamente': 0,
  'molto': 15290,
  'angoscia': 25,
  'angosciare': 0,
  'ansia': 1,
  'atterrire': 132,
  'panico': 15,
  'paura': 254,
  'raccapricciare': 10,
  'raccapriccio': 1,
  'schifo': 13,
  'sgomentare': 74,
  'sgomento': 8,
  'spaventare': 96,
  'spavento': 305,
  'terrore': 197,
  'domani': 45,
  'finora': 338,
  'ieri': 35,
  'oggi': 847,
  'presto': 382,
  'raramente': 10,
  'sempre': 2878,
  'spesso': 728},
 '1751-1800': {'atrocemente': 4,
  'formidabilmente': 0,
  'mostruosamente': 0,
  'orrendamente': 1,
  'orribilmente': 31,
  'spaventosamente': 1,
  'terribilmente': 17,
  'tremendamente': 0,
  'molto': 18597,
  'angoscia': 77,
  'angosciare': 0,
  'ansia': 2,
  'atterrire': 91,
  'panico': 11,
  'paura': 269,
  'raccapricciare': 2,
  'raccapriccio': 10,
  'schifo': 38,
  'sgomentare': 23,
  's

In [12]:
table = pd.DataFrame.from_dict(candidates, orient='index')
table

,atrocemente,formidabilmente,mostruosamente,orrendamente,orribilmente,spaventosamente,terribilmente,tremendamente,molto,angoscia,...,spavento,terrore,domani,finora,ieri,oggi,presto,raramente,sempre,spesso
1700-1750,2,0,2,1,28,13,17,0,15290,25,...,305,197,45,338,35,847,382,10,2878,728
1751-1800,4,0,0,1,31,1,17,0,18597,77,...,117,187,195,317,120,722,688,99,3595,1001
1801-1825,2,0,4,13,38,4,16,3,16435,150,...,147,224,413,144,611,2017,800,45,4668,1031
1826-1850,21,0,0,7,49,2,43,0,11535,147,...,308,351,168,172,135,827,611,25,2950,692
1851-1875,37,3,16,14,135,15,64,24,12718,232,...,223,350,547,220,295,1079,851,70,3473,771
1876-1900,74,15,10,37,160,35,96,37,12845,254,...,262,285,692,248,434,1080,842,184,4163,879
1901-1925,87,18,41,43,170,127,234,41,16377,462,...,335,457,1103,349,711,1447,1035,315,4718,1138
1926-1950,62,12,31,37,227,131,272,52,16335,435,...,286,496,1018,291,870,1857,733,313,4390,887
1951-1975,35,3,10,11,45,37,101,19,16800,220,...,96,260,317,155,455,1972,635,112,4199,1375
1976-2000,187,10,176,368,352,319,1383,552,21545,2807,...,1186,2663,2839,2506,4748,5205,2739,2321,6423,3295


In [13]:
table.to_csv('Korpus/candidates_after_lemmatization.csv', sep=';')